# Customer Segmentation using K-means Clustering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

In [ ]:

data = pd.read_excel('Online Retail.xlsx', engine='openpyxl')
print(data.head())
print(data.info())


  InvoiceNo StockCode                          Description  Quantity         InvoiceDate  UnitPrice  CustomerID         Country
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00       2.55     17850.0  United Kingdom
1    536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00       3.39     17850.0  United Kingdom
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00       2.75     17850.0  United Kingdom
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00       3.39     17850.0  United Kingdom
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00       3.39     17850.0  United Kingdom
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    5419

In [ ]:
if not data.empty:
    display(data.info())
    display(data.describe(include='all'))
    display(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


None

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,NaN,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,NaN,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,NaN
min,NaN,NaN,NaN,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,NaN
50%,NaN,NaN,NaN,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,NaN
75%,NaN,NaN,NaN,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,NaN
max,NaN,NaN,NaN,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,NaN


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:

if not data.empty:
 
    data = data.dropna(subset=['CustomerID'])
    if 'InvoiceNo' in data.columns:
        data = data[~data['InvoiceNo'].astype(str).str.startswith('C')]
    data = data.drop_duplicates()
    if 'InvoiceDate' in data.columns:
        data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')
    if {'Quantity','UnitPrice'}.issubset(data.columns):
        data['TotalAmount'] = data['Quantity'] * data['UnitPrice']
    display(data.head())
    print('Cleaned shape:', data.shape)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


Cleaned shape: (392732, 9)


In [ ]:
if not data.empty:

    data['CustomerID'] = data['CustomerID'].astype(str)
    snapshot_date = data['InvoiceDate'].max() + pd.Timedelta(days=1) if 'InvoiceDate' in data.columns and not data['InvoiceDate'].isnull().all() else pd.Timestamp.today()

    rfm = data.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
        'InvoiceNo': 'nunique',
        'TotalAmount': 'sum'
    }).reset_index()
    rfm.columns = ['CustomerID','Recency','Frequency','Monetary']
    display(rfm.head())
    print('RFM shape:', rfm.shape)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


RFM shape: (4339, 4)


In [ ]:
if 'rfm' in globals() and not rfm.empty:
    scaler = StandardScaler()
    rfm_scaled = rfm.copy()
    rfm_scaled[['Recency','Frequency','Monetary']] = scaler.fit_transform(rfm[['Recency','Frequency','Monetary']])
    display(rfm_scaled.head())
else:
    print('RFM not available to scale (check previous steps)')

,CustomerID,Recency,Frequency,Monetary
0,12346.0,2.334858,-0.424675,8.363977
1,12347.0,-0.905199,0.354080,0.251779
2,12348.0,-0.175186,-0.035297,-0.027938
3,12349.0,-0.735196,-0.424675,-0.032357
4,12350.0,2.174855,-0.424675,-0.190780


In [ ]:
output_csv = 'CustomerSegmentation_Features_AdityaRaj.csv'
if not data.empty:
    # Save RFM if exists, else save cleaned data
    if 'rfm' in globals() and not rfm.empty:
        rfm.to_csv(output_csv, index=False)
        print('Saved RFM features to', output_csv)
    else:
        data.to_csv('Cleaned_OnlineRetail_AdityaRaj.csv', index=False)
        print('Saved cleaned dataset to Cleaned_OnlineRetail_AdityaRaj.csv')
else:
    print('No data to save.')

Saved RFM features to CustomerSegmentation_Features_AdityaRaj.csv
